In [1]:
import sys

sys.path.insert(0, 'compiled_protobufs')

In [ ]:
# import IPython
# import sys
# from pygaggle.rerank.transformer import MonoT5
# !{sys.executable} -m pip install -r pygaggle/requirements.txt
# IPython.Application.instance().kernel.do_shutdown(True)  

In [ ]:
# """ Generate Taskgraphs from datasets """

# from models_datasets.recipe_1m_model import AbstractModelDataset
# from models_datasets.recipe_1m_model import Recipe1MModel
# from models_datasets.wikihow_model import WikihowModel

# dataset_models = {
#     "COOKING" : Recipe1MModel,
#     "DIY": WikihowModel,
# }

# for model_name, model in dataset_models.items():
#     dataset_model: AbstractModelDataset = model()
#     print(f"Dataset Model: {model_name}")
#     dataset_model.convert_to_taskgraphs()

In [3]:
import pandas as pd

""" Preprocess and load queries """

# from query_pipeline import query_pipeline
# query_pipeline.preprocess_queries()

queries = {
    "COOKING" : pd.read_csv('queries/cooking.csv'),
    "DIY": pd.read_csv('queries/diy.csv'),
}

queries["COOKING"].head()

,id,raw query,domain,knowledge,complex,many points of view,different key entities,seasonal,regional,target query,short query
0,query-0,how to spatchcock a turkey,cooking,n,y,n,n,y,usa,spatchcock turkey,spatchcock a turkey
1,query-1,I want an easy to make dessert for christmas,cooking,n,y,y,n,y,-,easi dessert christma,easy to make dessert for christmas
2,query-2,any recommendations for a gluten free appetizer,cooking,y,n,y,y,n,-,gluten free appet,gluten free appetizer
3,query-3,how to make a green goddess salad,cooking,y,y,n,y,n,usa,green goddess salad,green goddess salad
4,query-4,how to make a classic english trifle,cooking,y,n,n,y,y,uk,classic english trifl,classic english trifle


In [2]:
import numpy as np
from models_indexes.bm25_grid_search import GridSearchCV

best_params = {}

for domain in ["DIY", "COOKING"]:

    # Define the parameter grid to search over
    param_grid = {
        'k1': np.arange(0.1, 5.0, 0.1),
        'b': np.arange(0.1, 1.0, 0.1),
        'fb_terms': range(5, 101, 5),
        'fb_docs': range(5, 51, 5),
        'original_query_weight': np.arange(0.1, 1, 0.1),
    }

    grid_search = GridSearchCV(domain = domain, queries = queries[domain], params = param_grid, cv=5)
    best_params[domain] = grid_search.predict()

    print(f'Best parameters for {domain}: k1 = {grid_search.best_k1}, b = {grid_search.best_b}')

best_params

NameError: name 'queries' is not defined

In [7]:
best_params_bm25 = {
    'DIY': {'best_k1': 2.4, 'best_b': 0.58},
    'COOKING': {'best_k1': 0.62, 'best_b': 0.4}
}

best_params_bm25_rm3 = {
    'DIY': {'best_k1': 2.4, 'best_b': 0.58, 'best_fb_terms': 57.0, 'best_fb_docs': 9.0, 'best_original_query_weight': 0.48},
    'COOKING': {'best_k1': 0.62, 'best_b': 0.4, 'best_fb_terms': 88.0, 'best_fb_docs': 11.0, 'best_original_query_weight': 0.7}}

In [11]:
from models_indexes.bm25_model import BM25Model
# from models_indexes.ance_model import AnceModel
# from models_indexes.colbert_model import ColbertModel
from models_indexes.marqo_model import MarqoModel
# from models_indexes.abstract_model import AbstractModel
# from models_indexes.hybrid_model import HybridModel

models = {}

# for domain in  ["DIY"]:
for domain in  ["DIY", "COOKING"]:
    print(domain)
    models[domain] = {
        # "bm25" : BM25Model(domain = domain),
        # "bm25-tuned" : BM25Model(domain = domain, params = best_params[domain]),
        # "bm25+rm3" : BM25Model(domain = domain, rm3=True),
        # "bm25+t5" : BM25Model(domain = domain, t5=True),
        # "bm25+rm3-tuned" : BM25Model(domain = domain, rm3=True, params = best_params_bm25_rm3[domain]),
        # "ance+t5": AnceModel(domain = domain, t5=True),
        # "colbert+t5": ColbertModel(domain = domain, t5=True),
        "marqo": MarqoModel(domain = domain),
        # "marqo": MarqoModel(domain = domain),
        # "marqo": MarqoModel(domain = domain, doc_build="separate"),
        # "hybrid": HybridModel(domain = domain, rm3=True, t5=True)
    }

DIY
COOKING


In [14]:
# from models_indexes.marqo_model import MarqoModel
# model = MarqoModel(domain = "COOKING", t5 = False)
# qs = queries["COOKING"]

# # # model.convert_search_results_to_run(qs)
# # resp1 = model.search("pasta")
# # resp2 = model.search("fix kitchen",["Title"])
# # resp2

# # model.convert_search_results_to_run_attributes(qs, filters=["Tags"])
# # model.convert_search_results_to_run_attributes(qs, filters=["Tags", "Steps"])
# # model.convert_search_results_to_run_attributes(qs, filters=["Tags", "Steps", "Title"])
# model.convert_search_results_to_run_attributes(qs, filters=[ "Title", "Steps"])
# model.convert_search_results_to_run_attributes(qs, filters=[ "Title", "Requirements"])
# model.convert_search_results_to_run_attributes(qs, filters=[ "Title", "Steps" ,"Requirements"])
# model.convert_search_results_to_run_attributes(qs, filters=[ "Steps" ,"Requirements"])
# # model.convert_search_results_to_run_attributes(qs, filters=["Steps"])
# # model.convert_search_results_to_run_attributes(qs, filters=["Steps", "Title"])
# # model.convert_search_results_to_run_attributes(qs, filters=["Title"])


In [ ]:

# build indexes
# print("Build indexes")
# for domain, index_models in models.items():
#     for model_name, model in index_models.items():
#         model.build_index()

In [9]:
# create run files
print("Creating run files ")
for domain, index_models in models.items():
    qs = queries[domain]
    print(f"DOMAIN {domain}")
    for model_name, model in index_models.items():
        model.convert_search_results_to_run(qs)

Creating run files 
DOMAIN DIY
DOMAIN COOKING
